In [ ]:
from leip_recipe_designer import GoldenVolumes

## List the available Golden Volumes

What is a Golden Volume? 
- A comprehensive collection of recipes trained on a specific dataset.
- Evaluated on different hardwares.
- Detailed metrics captured for all these recipes on different hardwares and in-framework. 
- Detailed dataset information captured in a report. 

Each Volume contains - 
- Dataset Info 
- Golden recipes (Dataframe)

In the following few cells, you would see how to query different Golden Volumes Latent AI supports. (Continuously being added) It also displays information on a specific selected volume.

In [ ]:
goldenvolumes = GoldenVolumes()

In [ ]:
volumes = goldenvolumes.list_volumes_from_zoo()

### Look through the available Golden Datasets and select the one closest to yours (Eg. bdd100k dataset)

A summary of information about the dataset is shown in Jupyter Notebooks when the volume is placed as the last row in a cell as shown below. 

In [ ]:
volumes['bdd100k']

The methods used to create the above report are also available individually. For example:

- `volumes['bdd100k'].info` - returns a dictionary containing the summary info displayed on the "Info" tab.
- `volumes['bdd100k'].get_samples(split='train')` - shows an random subset of samples.
- `volumes['bdd100k'].boxes_info()` - A histogram of the number of bounding boxes for each image in the dataset.
- `volumes['bdd100k'].anchor_boxes()` - Plots the mean intersection over union (IoU) per each anchor box group (small, medium and large).
- `volumes['bdd100k'].class_distribution()` - Plots the mean intersection over union (IoU) per each anchor box group (small, medium and large).


In [ ]:
volumes['bdd100k'].boxes_info()

### Get the Golden Recipe Data frame from the selected Golden Dataset

Now we will extract the second part of a golden volume. i.e. The Golden Recipes in a dataframe. 
- Get a dataframe of tested recipes and their performance on hardware for the selected dataset.

In [ ]:
df = volumes['bdd100k'].get_golden_df()
df

What do these columns mean? To see all the columns of this DataFrame and a brief description of their data use `.describe_table()`

In [ ]:
volumes['bdd100k'].describe_table()

## Explore the Golden Recipe Dataframe to select the best recipes for your application/dataset

With all these recipes at your disposal, the following cells provides you with different tools to explore the dataframe according to your usecase. You can apply your Pandas skills to explore them. We share some examples below - 

In [ ]:
print('Number of recipes = ', len(df.index.get_level_values('id').unique()))

In [ ]:
## Query recipes with 'Device specific metrics' for Yolo v8 model on ['cuda:A4500' and 'cuda:AGX-Orin].
df.query("device in ['cuda:A4500', 'cuda:AGX-Orin'] and model_family=='Yolo v8'")

In [ ]:
## Query recipes with Quantization=None and width>512 with recipes sorted according to 'od_inf_rate' i.e. on device inference rate per ms.
df.query("quantization=='none' and width > 512").sort_values('od_inf_rate')

In [ ]:
## Query the best performing recipe based on 'if_task_metric'.
global_max = df['if_task_metric'].max()
print("The best performance (mAP) = ", global_max)

In [ ]:
## Sorting your recipes based on 'if_task_metric'
sub_df = df.sort_values(by=['if_task_metric'], ascending = False)
sub_df

In [ ]:
## Sorting your recipes based on 'od_inf_rate' for specific device and quantization.
sub_df2 = df.query("device=='cuda:A4500' and quantization=='none'").sort_values(by=['od_inf_rate'], ascending = True)
sub_df2

In [ ]:
## Now you can filter the above sub_df2 with the acceptable task_metric range and inference rate. 
sub_df3 = sub_df2.query('if_task_metric > 0.18 and if_task_metric < 0.21 and od_inf_rate < 12')
sub_df3

## Getting a recipe by selecting from a table and converting the SPPR.

Loading it from an SPPR - An SPPR is a serialized and portable representation of a recipe that is ready to execute. You can replace your dataset into the SPPR and start your training.

In [ ]:
from leip_recipe_designer.create import from_sppr
recipe = from_sppr(sub_df.query("device == 'cuda:A4500'").iloc[0]['sppr'])
recipe.describe()

In [ ]:
## Setting up your dataset. (Assuming your dataset is something like below.)

###################################################################################################
# from leip_recipe_designer.helpers.data import new_pascal_data_generator

# road_sign_data = new_pascal_data_generator(pantry=pantry,
#                                     root_path="your_data_path",
#                                     images_dir="images",
#                                     annotations_dir="annotations",
#                                     nclasses=4,
#                                     is_split=False,
#                                     trainval_split_ratio=0.80,
#                                     dataset_name="road-sign-data"
#                                     )
###################################################################################################

In [ ]:
## Replacing it with the selected SPPR

###################################################################################################
# from leip_recipe_designer.helpers.data import replace_data_generator
# replace_data_generator(recipe=recipe, data=road_sign_data)
###################################################################################################